# View: Revenue by Cities

## Data Source
- **Visits:** `workspace.hospital_silver.visits`
- **Patients:** `workspace.hospital_silver.patients`

## Details: 
- Location: `workspace.hospital_gold.view_patient_and_visits`
- Description: total revenue group by State and Cities

In [0]:
# Databricks Storage
catalog_name = "workspace"
schema_bronze = "hospital_bronze"
schema_silver = "hospital_silver"
schema_gold = "hospital_gold"

# view name: name of the view in schema and checkpoint
view_name = "view_revenue_by_cities"

# data source path
data_source = "s3://buckethospitaldata/data_streaming/"

# for streaming: schema and checkpoint location (stored in data source S3 buckets)
# checkpoint_location = f"{data_source}_checkpoints/view/{view_name}"

## Read Data from silver layer

In [0]:
df_visits = spark.read.table(f"{catalog_name}.{schema_silver}.visits")

In [0]:
df_patients = spark.read.table(f"{catalog_name}.{schema_silver}.patients")

In [0]:
df_cities = spark.read.table(f"{catalog_name}.{schema_silver}.cities")

## Join Datasets

In [0]:
df_join = df_visits.join(df_patients,df_visits.Patient_ID == df_patients.Patient_ID,"inner").join(df_cities,df_patients.City_ID == df_cities.City_ID,"inner")

## Aggregate Data

In [0]:
from pyspark.sql.functions import countDistinct, sum, approx_count_distinct

view_revenue_by_cities = df_join.groupBy("State", "City") \
    .agg(
        sum("Revenue_per_visit").alias("Revenue"),
        approx_count_distinct("visits.Patient_ID").alias("Number_of_unique_patient") 
    ) \
    .sort("State", "City")


## Write data as a View in Gold Layer

In [0]:
(
    view_revenue_by_cities.write
    .format("delta")
    .mode("overwrite")  
    .option("overwriteSchema", "true")  
    .saveAsTable(f"{catalog_name}.{schema_gold}.{view_name}")
)